In [1]:
import prody
import pandas as pd
import numpy as np
import os


In [2]:
def summarizeContacts(contacts):
    contact_data =[]
    for pair in contacts:
        atom1,atom2,dist = pair
        ch1 = atom1.getChid()
        ch2 = atom2.getChid()
        res1 = atom1.getResname()
        res2 = atom2.getResname()
        resnum1 = f"{atom1.getResnum()}{atom1.getIcode()}"
        resnum2 = f"{atom2.getResnum()}{atom2.getIcode()}"
        contact_data.append([ch1,res1,resnum1,ch2,res2,resnum2,dist])
    contact_data = pd.DataFrame(contact_data,columns=['ch1','res1','resnum1','ch2','res2','resnum2','dist'])
    #contact_data = contact_data.sort_values(['resnum1','resnum2','dist'],ascending=True).drop_duplicates(['resnum1','resnum2'])
    return contact_data

In [3]:
Nb_data = pd.read_excel("../SARS2_Nbs.xlsx")[['Name','pdb','VH Chain','RBD Chain']]
PanNb_data = pd.read_excel("../SARS2_PanNbs.xlsx")[['Name','pdb','VH Chain','RBD Chain']]

Nb_data = pd.concat([Nb_data,PanNb_data])

In [4]:
Nb_contact_summary = pd.read_csv("Nb_contact_summary.csv")

In [22]:
mutations= [339,346,371,373,375,417,440,446,452,477,478,484,493,496,498,501,505]

In [23]:
Omicron = [339,371,373,375,417,440,446,477,478,484,493,496,498,501,505]

In [5]:
additional_muts = [376,405,408]

# C alpha 8 A

In [6]:
all_hits=[]
all_nb_name = []
mutation_evaluated = additional_muts
for idx,(name,pdb,vh_chain,rbd_chain) in Nb_data[['Name','pdb','VH Chain','RBD Chain']].iterrows():
    #if not name in class1_nb_name:
    #    continue
    filename = pdb if ".pdb" in pdb else pdb+".pdb"
    filename = os.path.join("../Nbs",filename)
    
    if not os.path.exists(filename):
        continue
    struct = prody.parsePDB(filename)
    Ag = struct.select(f"protein chain {rbd_chain}")
    VH = struct.select(f"protein chain {vh_chain}")
    contacts= list(prody.iterNeighbors(Ag,4,VH))
    nb_epitope_res = set([i[0].getResnum() for i in contacts])
    
    mutation_hits=np.zeros(len(mutation_evaluated))
    
    overlap= set(mutation_evaluated)&nb_epitope_res
#     if len(overlap)==0:
#         print(name)
#         all_nb_name.append(name)
#         all_hits.append(mutation_hits)
#         continue
    
    summarized_contact = summarizeContacts(contacts)
    summarized_contact['resnum1'] = summarized_contact['resnum1'].astype(int)
    
    summarized_contact = summarized_contact[summarized_contact['resnum1'].apply(lambda x: x in overlap)]
    #print(summarized_contact)
    summarized_contact = dict(summarized_contact.groupby("resnum1")['resnum2'].count())
    for resnum,hit in summarized_contact.items():
        idx = mutation_evaluated.index(resnum)
        mutation_hits[idx] = hit
    all_nb_name.append(name)
    all_hits.append(mutation_hits)
    #print(nb_epitope_res)
    #overlap_variant = ",".join([str(j) for j in mutations&nb_epitope_res])
    #overlap_variant_num.append(len(overlap))
    #dis2.append(struct_distance)
    #print(name)
    #print(summarized_contact)
    #print([ACE2_centroid,nb_centroid],struct_distance)

@> 2852 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> Secondary structures were assigned to 176 residues.
@> 5111 atoms and 1 coordinate set(s) were parsed in 0.08s.
@> Secondary structures were assigned to 180 residues.
@> 3770 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> Secondary structures were assigned to 252 residues.
@> 2760 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> Secondary structures were assigned to 170 residues.
@> 15699 atoms and 1 coordinate set(s) were parsed in 0.18s.
@> Secondary structures were assigned to 1076 residues.
@> 3770 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> Secondary structures were assigned to 252 residues.
@> 2519 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> Secondary structures were assigned to 169 residues.
@> 4889 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> Secondary structures were assigned to 284 residues.
@> 53064 atoms and 1 coordinate set(s) were parsed in 0.55s.
@> Secondary stru

In [7]:
pd.DataFrame(all_hits,columns=mutation_evaluated,index=all_nb_name).to_csv("additional_VOC_nb_allatom_4.csv")

In [30]:
ab_seq_data = pd.read_excel("../SARS2_Abs.xlsx")

In [33]:
all_hits=[]
all_ab_name = []
mutation_evaluated = mutations

for idx,row in ab_seq_data.iterrows():
    pdb = row['pdb']
    name = row['Name']
    VH_chain = row['VH chain']
    VL_chain = row['VL chain']
    Ag_chain = row['RBD chain']
    filename = os.path.join("../Abs",f"{pdb.lower()}.pdb")
    #Ab_chain = f'chain {VH_chain} {VL_chain}'
    VH_chain = f'chain {VH_chain}'
    VL_chain = f'chain {VL_chain}'
    Ag_chain = f'chain {Ag_chain}'
    print(f'{filename}-{VH_chain}-{VL_chain}-{Ag_chain}')
    struct = prody.parsePDB(filename)
    
    VH_struct = struct.select(VH_chain).copy().ca
    Ag_struct = struct.select(Ag_chain).copy().ca
    contacts= list(prody.iterNeighbors(Ag_struct,8,VH_struct))
    
    
    nb_epitope_res = set([i[0].getResnum() for i in contacts])
    
    overlap= set(mutation_evaluated) & nb_epitope_res
    all_ab_name.append(name)
    mutation_hits=np.zeros(len(mutation_evaluated))
    if len(overlap)==0:
        all_hits.append(mutation_hits)
        continue
    
    
    
    summarized_contact = summarizeContacts(contacts)
    summarized_contact['resnum1'] = summarized_contact['resnum1'].astype(int)
    
    summarized_contact = summarized_contact[summarized_contact['resnum1'].apply(lambda x: x in overlap)]
    summarized_contact = dict(summarized_contact.groupby("resnum1")['resnum2'].count())
    
    for resnum,hit in summarized_contact.items():
        idx = mutation_evaluated.index(resnum)
        mutation_hits[idx] = hit
    all_hits.append(mutation_hits)
    #print(nb_epitope_res)
    #overlap_variant = ",".join([str(j) for j in Omicron&nb_epitope_res])
    #overlap_variant_num.append(len(overlap))
    #dis2.append(struct_distance)
    #print(name)
    #print(summarized_contact)
    #print([ACE2_centroid,nb_centroid],struct_distance)
pd.DataFrame(all_hits,columns=mutation_evaluated,index=all_ab_name).to_csv("VOC_Ab_ca_8.csv")


@> 4936 atoms and 1 coordinate set(s) were parsed in 0.06s.


../Abs/6w41.pdb-chain H-chain L-chain C
../Abs/6wps.pdb-chain H-chain L-chain A


@> 28116 atoms and 1 coordinate set(s) were parsed in 0.38s.
@> 9584 atoms and 1 coordinate set(s) were parsed in 0.10s.


../Abs/6xc2.pdb-chain H-chain L-chain A
../Abs/6xc4.pdb-chain H-chain L-chain A


@> 10309 atoms and 1 coordinate set(s) were parsed in 0.10s.


...

../Abs/7r8n.pdb-chain H-chain L-chain A


@> 29719 atoms and 1 coordinate set(s) were parsed in 0.27s.


../Abs/7r8o.pdb-chain H-chain L-chain B


@> 29952 atoms and 1 coordinate set(s) were parsed in 0.27s.
@> 2847 atoms and 1 coordinate set(s) were parsed in 0.03s.


../Abs/7ral.pdb-chain H-chain L-chain B


In [15]:
pd.DataFrame(all_hits,columns=Omicron,index=all_ab_name).to_csv("Omicron_ca.csv")